In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install kashgari==2.0.0

     |████████████████████████████████| 92kB 3.8MB/s 
     |████████████████████████████████| 51kB 4.7MB/s 
     |████████████████████████████████| 24.2MB 1.3MB/s 
     |████████████████████████████████| 378kB 38.7MB/s 
     |████████████████████████████████| 51kB 2.7MB/s 
  Created wheel for bert4keras: filename=bert4keras-0.9.7-cp36-none-any.whl size=43299 sha256=fab2cf86da9aede472ab56763fbf7fd657277a3d71bc0bf9e1ab1c5470f13582
  Stored in directory: /root/.cache/pip/wheels/71/7b/06/4b4bca2005cfccd3a157cb012d1f91a83c252442c9358c238c
Successfully built bert4keras
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
os.chdir("/content/drive/MyDrive/AiCup-1") #更改路徑
os.getcwd() #查看當前路徑

'/content/drive/MyDrive/AiCup-1'

In [ ]:
import numpy as np

file_path='test.txt'

def loadInputFile(path):
    trainingset = list()  # store trainingset [content,content,...]
    with open(file_path, 'r', encoding='utf8') as f:
        file_text=f.read().encode('utf-8').decode('utf-8-sig')
    datas=file_text.split('\n\n--------------------\n\n')[:-1]
    for data in datas:
        data=data.split('\n')
        content=data[1]
        trainingset.append(content)


    
    return trainingset

trainingset= loadInputFile(file_path)





In [ ]:
import kashgari
import re

loaded_model = kashgari.utils.load_model('BiLSTM_CRF_Model_2.h5')

labels = []
def cut_text(text, lenth):
    textArr = re.findall('.{' + str(lenth) + '}', text)
    textArr.append(text[(len(textArr) * lenth):])
    return textArr



def extract_labels(text, ners ,article):
    p=0
    ner_reg_list = []
    if ners:
        new_ners = []
        for ner in ners:
            new_ners += ner;
        for word, tag in zip([char for char in text], new_ners):
            if tag != 'O':
                ner_reg_list.append((word, tag , p))
            p+=1 

    #print(ner_reg_list)
    # 输出模型的NER识别结果
    #labels = {}
    
    if ner_reg_list:
        for i, item in enumerate(ner_reg_list):
            if item[1].startswith('B'):
                label = ""
                end = i + 1
                
                while end <= len(ner_reg_list) - 1 and ner_reg_list[end][1].startswith('I'):
                    end += 1

                ner_type = item[1].split('-')[1]
                label += ''.join([item[0] for item in ner_reg_list[i:end]])
                labels.append([article, item[2], int(item[2])+len(label) ,label, ner_type])

               
                
    return labels

for t in range(len(trainingset)):
    print("article:",t)
    text_input = trainingset[t]

    texts = cut_text(text_input, 100)

    ners = loaded_model.predict([[char for char in text] for text in texts])
    #print(ners)
    labels = extract_labels(text_input, ners , t)
    #print(labels)
    


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: The 'load_model' function is deprecated, use 'XX_Model.load_model' instead
  after removing the cwd from sys.path.
2020-12-28 12:55:32,543 [DEBUG] kashgari - ------------------------------------------------
2020-12-28 12:55:32,544 [DEBUG] kashgari - Loaded transformer model's vocab
2020-12-28 12:55:32,545 [DEBUG] kashgari - config_path       : chinese_L-12_H-768_A-12/bert_config.json
2020-12-28 12:55:32,546 [DEBUG] kashgari - vocab_path      : chinese_L-12_H-768_A-12/vocab.txt
2020-12-28 12:55:32,547 [DEBUG] kashgari - checkpoint_path : chinese_L-12_H-768_A-12/bert_model.ckpt
2020-12-28 12:55:32,548 [DEBUG] kashgari - Top 50 words    : ['[PAD]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]', '[unused9]', '[unused10]', '[unused11]', '[unused12]', '[unused13]', '[unused14]', '[unused15]', '[unused16]', '[unused17]', '[unused18]', '[unused19]', '[unu

{"class_name": "Functional", "config": {"name": "model_17", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, null], "dtype": "float32", "sparse": false, "ragged": false, "name": "Input-Token"}, "name": "Input-Token", "inbound_nodes": []}, {"class_name": "InputLayer", "config": {"batch_input_shape": [null, null], "dtype": "float32", "sparse": false, "ragged": false, "name": "Input-Segment"}, "name": "Input-Segment", "inbound_nodes": []}, {"class_name": "Embedding", "config": {"name": "Embedding-Token", "trainable": false, "batch_input_shape": [null, null], "dtype": "float32", "input_dim": 21128, "output_dim": 768, "embeddings_initializer": {"class_name": "TruncatedNormal", "config": {"mean": 0.0, "stddev": 0.02, "seed": null}}, "embeddings_regularizer": null, "activity_regularizer": null, "embeddings_constraint": null, "mask_zero": true, "input_length": null}, "name": "Embedding-Token", "inbound_nodes": [[["Input-Token", 0, 0, {}]]]}, {"class_name": "Embedd

2020-12-28 12:55:53,837 [WARNING] kashgari - Sequence length is None, will use the max length of the samples, which is 102
2020-12-28 12:55:53,839 [DEBUG] kashgari - predict seq_length: None, input: (2, 6, 102)


article: 0
None
1/1 [==============================] - 9s 9s/step


2020-12-28 12:56:02,426 [DEBUG] kashgari - predict output: (6, 102)
2020-12-28 12:56:02,431 [DEBUG] kashgari - predict output argmax: [[0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1 3 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 9 6 1 1 1 1 1 1
  1 1 9 6 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 2 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 1 

...

2020-12-28 13:11:57,687 [DEBUG] kashgari - predict output: (34, 102)
2020-12-28 13:11:57,690 [DEBUG] kashgari - predict output argmax: [[0 1 1 ... 1 1 0]
 [0 1 1 ... 1 1 0]
 [0 1 1 ... 1 1 0]
 ...
 [0 1 1 ... 1 1 0]
 [0 1 1 ... 1 1 0]
 [0 0 0 ... 0 0 0]]
2020-12-28 13:11:57,697 [DEBUG] kashgari - predict seq_length: None, input: (2, 39, 102)


article: 157
None
2/2 [==============================] - 14s 2s/step


2020-12-28 13:12:11,285 [DEBUG] kashgari - predict output: (39, 102)
2020-12-28 13:12:11,286 [DEBUG] kashgari - predict output argmax: [[0 1 1 ... 1 1 0]
 [0 1 1 ... 1 1 0]
 [0 1 1 ... 1 1 0]
 ...
 [0 1 1 ... 1 1 0]
 [0 1 1 ... 1 1 0]
 [0 1 1 ... 1 1 1]]
2020-12-28 13:12:11,293 [DEBUG] kashgari - predict seq_length: None, input: (2, 16, 102)


article: 158
None
1/1 [==============================] - 5s 5s/step


2020-12-28 13:12:16,826 [DEBUG] kashgari - predict output: (16, 102)
2020-12-28 13:12:16,828 [DEBUG] kashgari - predict output argmax: [[0 1 1 ... 1 1 0]
 [0 1 1 ... 1 1 0]
 [0 1 1 ... 1 1 0]
 ...
 [0 1 1 ... 1 1 0]
 [0 1 1 ... 1 1 0]
 [0 1 1 ... 1 1 1]]


In [ ]:
labels

[[0, 113, 115, '今天', 'time'],
 [0, 227, 229, '麻豆', 'location'],
 [0, 237, 239, '麻豆', 'location'],
 [0, 295, 297, '今年', 'time'],
 [1, 31, 33, '今天', 'time'],
 [1, 40, 42, '昨天', 'time'],
 [1, 76, 80, '0.79', 'med_exam'],
 [1, 94, 96, '88', 'med_exam'],
 [1, 122, 124, '60', 'med_exam'],
 [1, 182, 184, '19', 'med_exam'],
 [1, 258, 261, '2.6', 'med_exam'],
 [1, 360, 363, '黃醫師', 'name'],
 [1, 366, 369, '6.6', 'med_exam'],
 [1, 385, 388, '6.6', 'med_exam'],
 [1, 394, 397, '8.3', 'med_exam'],
 [1, 407, 410, '8.3', 'med_exam'],
 [1, 414, 418, '四個月前', 'time'],
 [1, 418, 421, '8.3', 'med_exam'],
 [1, 463, 466, '6.2', 'med_exam'],
 [1, 469, 472, '三個月', 'time'],
 [1, 497, 500, '劉老師', 'name'],
 [1, 626, 629, '黃醫師', 'name'],
 [1, 742, 744, '中午', 'time'],
 [1, 867, 869, '21', 'time'],
 [1, 873, 875, '21', 'time'],
 [1, 881, 884, '12天', 'time'],
 [1, 890, 893, '12天', 'time'],
 [1, 902, 906, '三個星期', 'time'],
 [2, 170, 174, '1100', 'med_exam'],
 [2, 245, 249, '1145', 'med_exam'],
 [2, 275, 278, '500', 'me

In [ ]:
import csv
with open('BiLSTM_CRF_Model_2.tsv', 'w', newline='') as tsvfile:
  writer = csv.writer(tsvfile)
  writer.writerow(['article_id','start_position','end_position','entity_text','entity_type'])
  # 寫入二維表格
  writer.writerows(labels)